In [1]:
import pandas as pd
import numpy as np
file = "Resources/purchase_data.csv"

df = pd.read_csv(file)
df.head()


,Purchase ID,SN,Age,Gender,Item ID,Item Name,Price
0,0,Lisim78,20,Male,108,"Extraction, Quickblade Of Trembling Hands",3.53
1,1,Lisovynya38,40,Male,143,Frenzied Scimitar,1.56
2,2,Ithergue48,24,Male,92,Final Critic,4.88
3,3,Chamassasya86,24,Male,100,Blindscythe,3.27
4,4,Iskosia90,23,Male,131,Fury,1.44


---
## Player Count
* Total Number of Players


In [2]:
np = df['SN'].value_counts().count()
tnp  = pd.DataFrame({'Total Players': [np]})
tnp.head()


,Total Players
0,576


---
## Purchasing Analysis (Total)
* Number of Unique Items
* Average Purchase Price
* Total Number of Purchases
* Total Revenue


In [3]:
# dataframe of Purchasing Analysis
pa = pd.DataFrame({'Number of Unique Items': [df['Item ID'].value_counts().count()] ,
                        'Average Purchase Price': [df['Price'].mean()] ,
                        'Total Number of Purchases': [df['Purchase ID'].count()],
                        'Total Revenue': [df['Price'].sum()]})

# change currency format for two columns:
pa['Average Purchase Price'] = pa['Average Purchase Price'].map("${:.2f}".format)
pa['Total Revenue'] = pa['Total Revenue'].map("${:.2f}".format)
pa.head()


,Number of Unique Items,Average Purchase Price,Total Number of Purchases,Total Revenue
0,183,$3.05,780,$2379.77


---
## Gender Demographics
* Percentage and Count of Male Players
* Percentage and Count of Female Players
* Percentage and Count of Other / Non-Disclosed


In [4]:
# dataframe: all players with their genders
df1 = df.groupby("SN").agg({"Gender":min})

# series: counting genders of all players
gd = df1["Gender"].value_counts()

# dataframe: all genders (total counts) of all players 
gd = gd.to_frame(name='Total Count')

# dataframe: all genders (total counts & percentages) of all players 
gd['Percentage of Players'] = round(gd['Total Count']/np*100,2)
gd.head()


,Total Count,Percentage of Players
Male,484,84.03
Female,81,14.06
Other / Non-Disclosed,11,1.91


---
## Purchasing Analysis (Gender)

* Purchase Count, Average Purchase Price, Total Purchase Value, and  Average Purchase Total per Person: by Gender


In [5]:
# dataframe: purchase count, avg. purchase price, total purchase value by gender
pag = df.groupby("Gender").agg({"Purchase ID": 'count' , "Price": ['mean','sum']})

# change column names and add in the column of 'Avg Purchase Total per Person'
pag.columns = ['Purchase Count','Avg Purchase Price','Total Purchase Value']
pag['Avg Purchase Total per Person'] = pag['Total Purchase Value']/gd['Total Count']

# # change currency format for the last three columns:
pag.iloc[:,1:]='$' + round(pag.iloc[:,1:],2).astype(str)
pag.head()


,Purchase Count,Avg Purchase Price,Total Purchase Value,Avg Purchase Total per Person
Gender,,,,
Female,113,$3.2,$361.94,$4.47
Male,652,$3.02,$1967.64,$4.07
Other / Non-Disclosed,15,$3.35,$50.19,$4.56


---
## Age Demographics

* Establish bins for ages


* Categorize the existing players using the age bins


* Calculate the numbers and percentages by age group


In [6]:
# bin number
nbin = 5

# Create bins in which to place values based upon Age
binList = [i for i in range(9,40, nbin)]
binList.insert(0,0)
binList.append(150)

# Create bin-labels
bin_labels =[str(binList[i]+1)+ '-' + str(binList[i+1]) for i in range(len(binList)-1)]
bin_labels[0]= '<' + str(binList[1]+1)
bin_labels[-1]= str(binList[-2]+1) + '+'

# OPTIONAL: make a backup for the original dataframe
df_bk = df

# add new column of Age bins into the original dataframe
df['Age_Bin'] = pd.cut(df['Age'],binList,labels=bin_labels)

# dataframe: all players with their ages (in bins)
df2 = df.groupby("SN").agg({"Age_Bin": 'min'})

# series: counting ages (in bins) of all players
ab = df2["Age_Bin"].value_counts()

# dataframe: all ages (in bins) of all players (in sorted order)
ab = ab.to_frame(name='Total Count').reindex(bin_labels)
ab['Percentage of Players'] = round(ab['Total Count']/np*100,2)
ab


,Total Count,Percentage of Players
<10,17,2.95
10-14,22,3.82
15-19,107,18.58
20-24,258,44.79
25-29,77,13.37
30-34,52,9.03
35-39,31,5.38
40+,12,2.08


---
## Purchasing Analysis (Age)

* The below each broken into bins of 4 years (i.e. <10, 10-14, 15-19, etc.)
 * Purchase Count
 * Average Purchase Price
 * Total Purchase Value
 * Average Purchase Total per Person by Age Group
 

In [7]:
# dataframe: purchase count, avg. purchase price, total purchase value by gender
paa = df.groupby("Age_Bin").agg({"Purchase ID": 'count' , "Price": ['mean','sum']})

# change column names and add in the column of 'Avg Purchase Total per Person'
paa.columns = ['Purchase Count','Avg Purchase Price','Total Purchase Value']
paa['Avg Purchase Total per Person'] = paa['Total Purchase Value']/ab['Total Count']

# change currency format for the last three columns:
paa.iloc[:,1:]='$' + round(paa.iloc[:,1:],2).astype(str)
paa


,Purchase Count,Avg Purchase Price,Total Purchase Value,Avg Purchase Total per Person
Age_Bin,,,,
<10,23,$3.35,$77.13,$4.54
10-14,28,$2.96,$82.78,$3.76
15-19,136,$3.04,$412.89,$3.86
20-24,365,$3.05,$1114.06,$4.32
25-29,101,$2.9,$293.0,$3.81
30-34,73,$2.93,$214.0,$4.12
35-39,41,$3.6,$147.67,$4.76
40+,13,$2.94,$38.24,$3.19


---
## Top Spenders

* Identify the the top 5 spenders in the game by total purchase value, then list (in a table):
 * SN
 * Purchase Count
 * Average Purchase Price
 * Total Purchase Value
 


In [8]:
# dataframe: purchase count and total purchase value by each player, SORTED by total purchase values
df1 = df.groupby("SN").agg({"Purchase ID": 'count',"Price": 'sum'}).sort_values('Price',ascending=False)

# insert the column of 'Average Purchase Price' into the dataframe
df1.insert(loc=1,column='Average Purchase Price', value=df1['Price']/df1['Purchase ID'])
df1.columns= ['Purchase Count','Average Purchase Price','Total Purchase Value']

# change currency format for the last two columns:
df1.iloc[:,1:]='$' + round(df1.iloc[:,1:],2).astype(str)
df1.head()



,Purchase Count,Average Purchase Price,Total Purchase Value
SN,,,
Lisosia93,5,$3.79,$18.96
Idastidru52,4,$3.86,$15.45
Chamjask73,3,$4.61,$13.83
Iral74,4,$3.4,$13.62
Iskadarya95,3,$4.37,$13.1


---
## Most Popular Items

* Identify the 5 most popular items by purchase count, then list (in a table):
 * Item ID
 * Item Name
 * Item Price
 * Total Purchase Value
 


In [9]:
# dataframe: item ID, item name, and item price of each item, 
df1 = df.groupby("Item ID").agg({"Item Name": 'min',"Purchase ID": 'count', "Price":'min'})

# insert the column of 'Total Purchase Value' into the dataframe
df1['Total Purchase Value'] = df1['Purchase ID']*df1['Price']
df1.columns= ['Item Name','Purchase Count','Item Price','Total Purchase Value']

# SORTING dataframe by total purchase values
df1Sorted = df1.sort_values('Purchase Count',ascending=False)

# change currency format for the last two columns:
df1Sorted.iloc[:,2:]='$' + round(df1Sorted.iloc[:,2:],2).astype(str)

df1Sorted.head()


,Item Name,Purchase Count,Item Price,Total Purchase Value
Item ID,,,,
178,"Oathbreaker, Last Hope of the Breaking Storm",12,$4.23,$50.76
145,Fiery Glass Crusader,9,$4.58,$41.22
108,"Extraction, Quickblade Of Trembling Hands",9,$3.53,$31.77
82,Nirvana,9,$4.9,$44.1
19,"Pursuit, Cudgel of Necromancy",8,$1.02,$8.16


---
## Most Profitable Items

* Identify the 5 most profitable items by total purchase value, then list (in a table):
 * Item ID
 * Item Name
 * Item Price
 * Total Purchase Value
 


In [19]:
# SORTING dataframe by total purchase values
df2Sorted = df1.sort_values('Total Purchase Value',ascending=False)

# change currency format for the last two columns:
df2Sorted.iloc[:,2:]='$' + round(df2Sorted.iloc[:,2:],2).astype(str)

df2Sorted.head()


,Item Name,Purchase Count,Item Price,Total Purchase Value
Item ID,,,,
178,"Oathbreaker, Last Hope of the Breaking Storm",12,$4.23,$50.76
82,Nirvana,9,$4.9,$44.1
145,Fiery Glass Crusader,9,$4.58,$41.22
92,Final Critic,8,$4.88,$39.04
103,Singed Scalpel,8,$4.35,$34.8


---
## Three observable trends based on the data:

1. Male players tends to purchase more optional items than other genders.

2. Young people, especially who are from 15-29, are top spenders.

3. Items with the high frequently purchase often make high profitable values, except for those whose prices are too low.
